# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import json
import os
import os.path as _p
import sys
import datetime
import slugify
import hashlib
from fs.osfs import OSFS
from pathlib import Path
from schematized_config.core import ConfigValidator
from blobslayer.DotenvSchema import DotenvSchema

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
def recursive_substitute(env_vars):
    """
    Recursively substitute environment variables in the values of the env_vars dictionary.
    """
    while True:
        changed = False
        for key, value in env_vars.items():
            substituted_value = value
            for k, v in env_vars.items():
                # Substitute the value if the key is found in the value string
                if f"${k}" in substituted_value:
                    substituted_value = substituted_value.replace(f"${k}", v)
                    changed = True
            
            env_vars[key] = substituted_value

        # If no substitutions were made in this pass, break the loop
        if not changed:
            break

    return env_vars

In [ ]:
#| hide

# REMOVE ME
try:
    cwd = os.getcwd()
    os.chdir('..')
    os.environ['CONFIG_VALIDATOR_JSON_SCHEMA'] = './schemas/dotenv.schema.json'
    dotenv_path = 'dotenv'
    if _p.exists(dotenv_path) and 'CONFIG_VALIDATOR_JSON_SCHEMA' in os.environ:
        validated_dotenv = ConfigValidator.load_dotenv(
            dotenv_path=dotenv_path,
            storage_driver=OSFS('.'),
        )  # use defaults of .env and CONFIG_VALIDATOR_JSON_SCHEMA

    os.chdir('nbs')
except Exception as e:
    print('failed to load env:')
    print(e)
    os.chdir(cwd)

Config = DotenvSchema(**recursive_substitute(validated_dotenv))

In [ ]:
Config.STORAGE_DIRECTORY

'/mnt/c/Users/Alex Huang/Desktop/work/_blobslayer-storage'

In [ ]:
def ensure_storage_directories_exist():    
    Path(Config.STORAGE_DIRECTORY).mkdir(parents=True, exist_ok=True)
    Path(Config.BLOBS_DIRECTORY).mkdir(parents=True, exist_ok=True)
    Path(Config.METADATA_DIRECTORY).mkdir(parents=True, exist_ok=True)


In [ ]:
ensure_storage_directories_exist()


In [ ]:
def sha256sum(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

def to_split_path(hash_path: str) -> str:
    return f'{hash_path[:2]}/{hash_path[2:4]}/{hash_path[4:]}'

def save_blob(data: bytes) -> str:
    blob_key = sha256sum(data)
    blob_path = to_split_path(blob_key)
    blob_split_dir, blob_split_filename = _p.split(blob_path)
    parent_dir = Path(Config.BLOBS_DIRECTORY) / blob_split_dir
    parent_dir.mkdir(parents=True, exist_ok=True)
    with OSFS(parent_dir) as myfs:
        if myfs.exists(blob_split_filename):
            return blob_key
        with myfs.open(blob_split_filename, 'wb') as ofile:
            ofile.write(data)
    return blob_key

def get_blob(blob_key: str):  # returns a file-like
    blob_path = to_split_path(blob_key)
    blob_split_dir, blob_split_filename = _p.split(blob_path)
    parent_dir = Path(Config.BLOBS_DIRECTORY) / blob_split_dir
    with OSFS(parent_dir) as myfs:
        with myfs.open(blob_split_filename, 'rb') as ifile:
            return ifile


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()